In [1]:
import pandas as pd

import numpy as np

import holoviews as hv
import plotly.graph_objects as go
import plotly.express as pex

In [2]:
df_raw = pd.read_csv('data/goemotions.csv')

In [4]:
df_raw

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1548381039,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1548084169,37,True,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1546427744,37,False,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1547965054,18,False,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1546668601,2,False,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211220,Everyone likes [NAME].,ee6pagw,Senshado,heroesofthestorm,t3_agjf24,t3_agjf24,1547634322,16,False,0,...,1,0,0,0,0,0,0,0,0,0
211221,Well when you’ve imported about a gazillion of...,ef28nod,5inchloser,nottheonion,t3_ak26t3,t3_ak26t3,1548553182,15,False,0,...,0,0,0,0,0,0,0,0,0,0
211222,That looks amazing,ee8hse1,springt1me,shittyfoodporn,t3_agrnqb,t3_agrnqb,1547683734,70,False,1,...,0,0,0,0,0,0,0,0,0,0
211223,The FDA has plenty to criticize. But like here...,edrhoxh,enamedata,medicine,t3_aejqzd,t1_edrgdtx,1547168832,4,False,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
authors_1 = df_raw.loc[lambda f: f['subreddit']=='Advice']['author'].drop_duplicates()

In [22]:
authors_2 = df_raw.loc[lambda f: f['subreddit']=='doordash']['author'].drop_duplicates().sort_values()

In [23]:
pd.merge(authors_1, authors_2, how='inner')

,author
0,[deleted]
1,BlowingSmokeUpYourAs


In [31]:
df_raw.loc[lambda f: f['author'] == '[deleted]']

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
10,"I have, and now that you mention it, I think t...",ed9w1hm,[deleted],AskMenOver30,t3_acm7f6,t1_ed9m2l0,1546658263,36,False,0,...,0,0,0,0,0,0,0,0,0,1
14,Build a wall? /jk,edsqvyx,[deleted],AskMenOver30,t3_aeq3ep,t1_edsn1bd,1547207519,10,False,0,...,0,0,0,0,0,0,0,0,0,1
92,I didn't like the family ending but I liked th...,ee6mb6z,[deleted],assassinscreed,t3_agboi2,t1_ee6m488,1547628831,56,False,0,...,0,0,0,0,0,0,0,0,0,0
107,My fans on patreon will be rewarded soon,ef8s6ku,[deleted],nrl,t3_akwycz,t1_ef8pvmo,1548746680,33,False,0,...,0,0,0,0,0,0,0,0,0,1
161,"Integrated, Antwaarps speaking, second generat...",eepltiz,[deleted],belgium,t3_ai8u8j,t1_eem1uan,1548184008,34,False,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210815,This post is inappropriate for this subreddit.,edau7nd,[deleted],SubredditDrama,t3_acuh2o,t3_acuh2o,1546698714,39,False,0,...,0,0,0,0,0,0,0,0,0,0
210886,This post is inappropriate for this subreddit.,edau7nd,[deleted],SubredditDrama,t3_acuh2o,t3_acuh2o,1546698714,26,False,0,...,0,0,0,0,0,0,0,0,0,0
210899,This post is inappropriate for this subreddit.,edau7nd,[deleted],SubredditDrama,t3_acuh2o,t3_acuh2o,1546698714,37,False,0,...,0,0,0,0,1,0,0,0,0,0
210934,Fcking disgusting.,edwux5m,[deleted],COMPLETEANARCHY,t3_af8jgr,t3_af8jgr,1547319348,35,False,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
emotions_cols = ['admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral']

In [4]:
emotion_categories = {'admiration': 'Positive', # Proposed simplified classification
                      'amusement':'Positive',
                      'anger':'Negative',
                      'annoyance':'Negative',
                      'approval':'Positive',
                      'caring':'Positive',
                      'confusion':'Neutral',
                      'curiosity':'Positive',
                      'desire':'Positive',
                      'disappointment':'Negative',
                      'disapproval':'Negative',
                      'disgust':'Negative',
                      'embarrassment':'Neutral',
                      'excitement':'Positive',
                      'fear':'Negative',
                      'gratitude':'Positive',
                      'grief':'Negative',
                      'joy':'Positive',
                      'love':'Positive',
                      'nervousness':'Neutral',
                      'optimism':'Positive',
                      'pride':'Positive',
                      'realization':'Neutral',
                      'relief':'Positive',
                      'remorse':'Negative',
                      'sadness':'Negative',
                      'surprise':'Neutral',
                      'neutral':'Neutral'
                       }

In [14]:
df = df_raw.groupby(['id', 'subreddit', 'text'])[emotions_cols].sum().reset_index() # Aggregates by post

In [6]:
# Only keeps emotion(s) with the most votes for each post
"""
df['max_val'] = df[emotions_cols].max(axis=1)
for col in emotions_cols:
    df[col] = df[col] - df['max_val']
del df['max_val']
df_new = (df.melt(['id', 'subreddit'], var_name='emotion').query('value >= 0')
       .groupby(['id', 'subreddit'])['emotion']
       .apply(', '.join)
       .reset_index())
df_grouped = df_new.groupby(['subreddit', 'emotion']).count().reset_index()
"""

"\ndf['max_val'] = df[emotions_cols].max(axis=1)\nfor col in emotions_cols:\n    df[col] = df[col] - df['max_val']\ndel df['max_val']\ndf_new = (df.melt(['id', 'subreddit'], var_name='emotion').query('value >= 0')\n       .groupby(['id', 'subreddit'])['emotion']\n       .apply(', '.join)\n       .reset_index())\ndf_grouped = df_new.groupby(['subreddit', 'emotion']).count().reset_index()\n"

In [15]:
df.groupby(['subreddit']).count().sort_values('id')

,id,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
subreddit,,,,,,,,,,,,,,,,,,,,,
farcry,25,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25
shieldbro,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,32,32,32,32,32,32
Anticonsumption,34,34,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
SSBM,35,35,35,35,35,35,35,35,35,35,...,35,35,35,35,35,35,35,35,35,35
darknet,38,38,38,38,38,38,38,38,38,38,...,38,38,38,38,38,38,38,38,38,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vanderpumprules,221,221,221,221,221,221,221,221,221,221,...,221,221,221,221,221,221,221,221,221,221
AnimalsBeingBros,231,231,231,231,231,231,231,231,231,231,...,231,231,231,231,231,231,231,231,231,231
socialanxiety,232,232,232,232,232,232,232,232,232,232,...,232,232,232,232,232,232,232,232,232,232


In [20]:
subreddit_df = pd.DataFrame(columns = ['subreddit', 'emotion', 'count'])
for emotion in emotions_cols:
    gp_df = df.groupby(['subreddit'])[emotion].sum()
    d = {'subreddit': gp_df.index, 'emotion': [emotion]*len(gp_df), 'count':gp_df.values}
    subreddit_df = subreddit_df.append(pd.DataFrame(data = d), ignore_index=True)

In [9]:
#hv.extension('bokeh')
#hv.Sankey(subreddit_df.loc[lambda f: f['subreddit'] == 'socialanxiety'])

In [59]:
#subreddit_df.to_csv('subreddit_emotion.csv')

In [39]:
text_df = pd.DataFrame(columns = ['subreddit', 'emotion', 'text'])
for emotion in emotions_cols:
    df_long = df[df['text'].str.split().str.len().ge(4)] # Remove short posts
    df_sorted = df_long.sort_values(emotion, ascending=False)
    gp_df = df_sorted.groupby('subreddit').first()
    d = {'subreddit': gp_df.index, 'emotion': [emotion]*len(gp_df), 'text':gp_df['text']}
    text_df = text_df.append(pd.DataFrame(data = d), ignore_index=True)

In [44]:
full_df = pd.merge(text_df, subreddit_df, how='left', on=['subreddit', 'emotion'])

In [46]:
full_df.columns = ['source', 'target', 'text', 'value']

In [47]:
full_df.to_csv('subreddit_emotion_2.csv')

In [45]:
full_df

,subreddit,emotion,text,count
0,2meirl4meirl,admiration,Or embalmed! Green burial is the best burial,47
1,49ers,admiration,Can we please just call out defense next year ...,50
2,4PanelCringe,admiration,Seriously. [NAME] is a hero. I heard he was a ...,52
3,90DayFiance,admiration,maybe. Id praise her for it though. In a world...,76
4,90dayfianceuncensored,admiration,She could and should pursue modeling! It seeme...,43
...,...,...,...,...
13519,worldpolitics,neutral,The original video was debunked. Do you agree ...,156
13520,yesyesyesyesno,neutral,This is like 4 years old from vine,198
13521,youseeingthisshit,neutral,Imagine being upset over who someone loves.,198
13522,youtube,neutral,Naming comes from the heart,95
